In [1]:
# Sort based on match scores, 
# 1. cutoff, 2. save file with annotation
import collections

import compress_pickle
import itertools
import libsbml
import numpy as np
import os
import pickle
import pandas as pd
import sys
import time
import matplotlib.pyplot as plt
%matplotlib inline  

BIOMD_12 = 'BIOMD0000000012.xml'
BASE_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/'
DATA_DIR = os.path.join(BASE_DIR, "DATA")
ALGO_DIR = os.path.join(DATA_DIR, "algo")
CHEBI_DIR = os.path.join(DATA_DIR, "chebi")
RHEA_DIR = os.path.join(DATA_DIR, "rhea")
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
BIGG_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/DATA/bigg'
ecoli_fpath = os.path.join(BIGG_DIR, "e_coli_core.xml")

PROJ_DIR = os.path.join(os.getcwd(), os.pardir)
AMAS_DIR = os.path.join(PROJ_DIR, "AMAS")
sys.path.append(PROJ_DIR)

SUPPL_DIR = os.path.join(PROJ_DIR, os.pardir, "AMAS_suppl")
ACCURACY_DIR = os.path.join(SUPPL_DIR, "data_for_credibility")
RESULT_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/AMAS_suppl/result_files'

from AMAS import species_annotation as sa
from AMAS import reaction_annotation as ra
from AMAS import recommender
from AMAS import constants as cn
from AMAS import iterator as it
from AMAS import tools

In [2]:
BIOMD_190_PATH = os.path.join(cn.TEST_DIR, 'BIOMD0000000190.xml')
recom = recommender.Recommender(libsbml_fpath=BIOMD_190_PATH)
one_recom = recom.getSpeciesAnnotation(pred_id='SAM')
two_recom = recom.getReactionAnnotation(pred_id='VCoA')

In [3]:
filt_recom = recom.filterRecommendationByThreshold(inp_recom=one_recom,
                                                   inp_thresh=0.8)
filt_recom.candidates

[('CHEBI:15414', 1.0), ('CHEBI:59789', 1.0)]

In [9]:
filt_recom.candidates

[('CHEBI:15414', 1.0), ('CHEBI:59789', 1.0)]

In [12]:
from AMAS import annotation_maker as am

maker = am.AnnotationMaker('species', '00001')
res = maker.getAnnotationString(filt_recom.candidates)
print(res)

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:vcard4="http://www.w3.org/2006/vcard/ns#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">
    <rdf:Description rdf:about="#metaid_00001">
      <bqbiol:is>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:15414"/>
          <bqbiol:hasProperty>
            <rdf:Bag>
              <rdf:li rdf:resource="http://amas/match_score/by_name/1.0"/>
            </rdf:Bag>
          </bqbiol:hasProperty>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:59789"/>
          <bqbiol:hasProperty>
            <rdf:Bag>
              <rdf:li rdf:resource="http://amas/match_score/by_name/1.0"/>
            </rdf:Bag>
          </bqbiol:hasProperty>
        </rdf:Bag>
      </bqbiol:is>
    </rdf:Description>
  </rdf:RDF>
</annotation>


In [5]:
model = recom.sbml_document.getModel()
one_s = model.getSpecies('SAM')
print(one_s.getAnnotationString())

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:bqmodel="http://biomodels.net/model-qualifiers/" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/">
    <rdf:Description rdf:about="#metaid_0000036">
      <bqbiol:is>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/obo.chebi/CHEBI:15414"/>
          <rdf:li rdf:resource="http://identifiers.org/kegg.compound/C00019"/>
        </rdf:Bag>
      </bqbiol:is>
    </rdf:Description>
  </rdf:RDF>
</annotation>


In [6]:
one_s.setAnnotation(res)

0

In [7]:
print(one_s.getAnnotationString())

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:vcard4="http://www.w3.org/2006/vcard/ns#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">
    <rdf:Description rdf:about="#metaid_0000036">
      <bqbiol:is>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:15414"/>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:59789"/>
        </rdf:Bag>
      </bqbiol:is>
    </rdf:Description>
    <rdf:Description rdf:about="#metaid_not_provided">
      <bqbiol:is>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:15414"/>
          <bqbiol:hasProperty>
            <rdf:Bag>
              <rdf:li rdf:resource="http://amas/match_score/by_name/1.0"/>
            </rdf:Bag>
          </bqbiol:hasProperty>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:59

In [13]:
maker.createTag('annotation', 0)

['<annotation>', '</annotation>']

In [17]:
with open(os.path.join(cn.TEST_DIR, 'annotation_example.txt')) as file:
    lines = [line.rstrip() for line in file]

In [18]:
lines

['<annotation>',
 '  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:vcard4="http://www.w3.org/2006/vcard/ns#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">',
 '    <rdf:Description rdf:about="#metaid_00001">',
 '      <bqbiol:is>',
 '        <rdf:Bag>',
 '          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:15414"/>',
 '          <bqbiol:hasProperty>',
 '            <rdf:Bag>',
 '              <rdf:li rdf:resource="http://amas/match_score/by_name/1.0"/>',
 '            </rdf:Bag>',
 '          </bqbiol:hasProperty>',
 '          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:59789"/>',
 '          <bqbiol:hasProperty>',
 '            <rdf:Bag>',
 '              <rdf:li rdf:resource="http://amas/match_score/by_name/1.0"/>',
 '            </rdf:Bag>',
 '          </bqbiol:hasProperty>',
 '        </rdf:Bag>',
 '      </bqbiol:is>',
 '

In [24]:
block = maker.createAnnotationBlock(identifier='CHEBI:15414',
                            match_score=1.0,
                            knowledge_resource=None,
                            score_type=None,
                            nested_prefix='bqbiol:hasProperty')

In [27]:
print('\n'.join(block))

<rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:15414"/>
<bqbiol:hasProperty>
  <rdf:Bag>
    <rdf:li rdf:resource="http://amas/match_score/by_name/1.0"/>
  </rdf:Bag>
</bqbiol:hasProperty>


In [30]:
print(block)
s = block.pop()
print(s)
print(block)

['<rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:15414"/>', '<bqbiol:hasProperty>', '  <rdf:Bag>', '    <rdf:li rdf:resource="http://amas/match_score/by_name/1.0"/>', '  </rdf:Bag>']
  </rdf:Bag>
['<rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:15414"/>', '<bqbiol:hasProperty>', '  <rdf:Bag>', '    <rdf:li rdf:resource="http://amas/match_score/by_name/1.0"/>']


In [32]:
maker._createAnnotationContainer(['annotation'])

['<annotation>', '</annotation>']

In [ ]:
maker.getAnnotationString